# Azure CLI Account Setup for Azure Resource Manager (ARM)

Azure CLI provides an easy way to setup an account for Azure Resource Manager (ARM) and furthermore creates an new service principal for the Simple Azure access. In this tutorial, we use IPython helper (!) to run Azure CLI. 

## Interactive Login Azure Portal

azure cli tool asks you to open a web browser and sign in the azure portal to authenciate. The following command ``azure login`` will guide you to the page https://aka.ms/devicelogin with a unique one-time login verification code.
You will be asked to type the code in your browser to complete the login. 

**NOTE** Run all cells step-by-step with instructions to complete Azure Account Setup.

In [2]:
!yes|azure login

Microsoft Azure CLI would like to collect data about how users use CLI
commands and some problems they encounter.  Microsoft uses this information
to improve our CLI commands.  Participation is voluntary and when you
choose to participate your device automatically sends information to
Microsoft about how you use Azure CLI. 

If you choose to participate, you can stop at any time later by using Azure
CLI as follows: 
1.  Use the azure telemetry command to turn the feature Off. 
To disable data collection, execute: azure telemetry --disable

If you choose to not participate, you can enable at any time later by using
Azure CLI as follows: 
1.  Use the azure telemetry command to turn the feature On. 
To enable data collection, execute: azure telemetry --enable

Select y to enable data collection :(y/n) 
You choose to participate in Microsoft Azure CLI data collection.


info:    Executing command login
\info:    To sign in, use a web browser to open the page https://aka.ms/devicelogin. Ent

## Credentials for Azure Python SDK

Azure Python SDK which Simple Azure is based on requires the credential information below for ARM and ASM (Azure Service Management).

- subscription ID
- tenant ID
- client ID
- client secret

The following sections demonstrate Azure CLI commands to obtain these information step-by-step.

### Subscription ID and Tenant ID

``account show`` displays subscription id and tenant id as **ID** and **Tenant ID**.

In [4]:
!azure account show

info:    Executing command account show
data:    Name                        : Simple-Azure-Subscription
data:    ID                          : 5s3ag2s5-2aa1-4828-xxxx-9g8sw72w5w5g
data:    State                       : Enabled
data:    Tenant ID                   : 5e39a20e-c55a-53de-xxxx-2503a55et6ta
data:    Is Default                  : true
data:    Environment                 : AzureCloud
data:    Has Certificate             : No
data:    Has Access Token            : Yes
data:    User name                   : hroe.lee@gmail.com
data:    
info:    account show command OK


IPython filters the subscription ID and tenant ID using ``awk`` command and stores into **sid** and **tid** variables.

In [ ]:
sid_tid = !azure account show|awk -F ':' '/ID/{ print $3}'
sid = sid_tid[0]
tid = sid_tid[1]

### Service Principal for Simple Azure

Once you loaded your azure credential, a service principal is required to get access of resource groups therefore Azure Services via Azure Resource Manager and Templates are permitted to use in Simple Azure. Azure CLI provides a few commands to complete this step.

"azure ad sp create" command create a new service principal in Active Directory with a name (--name option).

In [10]:
out=!azure ad sp create --name simpleazure
cid = out[6].split(":")[1].lstrip()
newout="\n".join(out)
print(newout)

info:    Executing command ad sp create
+
+
data:    Object Id:               5d79f365-26e0-4993-8fe7-7021b3fd373d
data:    Display Name:            simpleazure
data:    Service Principal Names:
data:                             ca66450a-2532-4e8b-81b2-31722d36d808
data:                             http://simpleazure
info:    ad sp create command OK


``Id`` after Service Principal Names is our **client id** for Simple Azure. ``cid`` variable stores the ID in the previous commands.

#### Set a Password for Service Principal

A password for Service Principal will be used as **client_secret** later in Simple Azure.
Please provide your desired password in below.

In [72]:
password=""

In [19]:
!azure ad sp set -p $password $cid

info:    Executing command ad sp set
+
data:    Object Id:               5d79f365-26e0-4993-8fe7-7021b3fd373d
data:    Display Name:            simpleazure
data:    Service Principal Names:
data:                             http://simpleazure
data:                             ca66450a-2532-4e8b-81b2-31722d36d808
info:    ad sp set command OK


Note that '$cid' is a client id obtained from the previous command.

#### Assign Role to Service Principal

Assigning role permits certain actions to your service principal under your subscription id. "Owner" allows you have every rights to use resources without restrictions. See more roles: [here](https://azure.microsoft.com/en-us/documentation/articles/role-based-access-built-in-roles/)

In [62]:
!azure role assignment create --objectId $cid -o Owner -c /subscriptions/$sid

info:    Executing command role assignment create
+
\data:    RoleAssignmentId     : /subscriptions/5s3ag2s5-2aa1-4828-xxxx-9g8sw72w5w5g/providers/Microsoft.Authorization/roleAssignments/6a51ba07-5470-4e09-875c-0d0362e46901
data:    RoleDefinitionName   : Owner
data:    RoleDefinitionId     : 8e3af657-a8ff-443c-a75c-2fe8c4bcb635
data:    Scope                : /subscriptions/5s3ag2s5-2aa1-4828-xxxx-9g8sw72w5w5g
data:    Display Name         : simpleazure
data:    SignInName           : undefined
data:    ObjectId             : 5d79f365-26e0-4993-8fe7-7021b3fd373d
data:    ObjectType           : ServicePrincipal
data:    
+
info:    role assignment create command OK


Are you completed all steps without any issues?
Congraturations! You just completed login setup for your azure account.

## Sample Run Simple Azure after Login Setup

Let's try to deploy a sample template using Simple Azure and the credentials that we just obtained.

### Import Simple Azure

In [42]:
from  simpleazure import SimpleAzure as saz

### Credentials via Environment Variables

In [38]:
import os
os.environ['AZURE_SUBSCRIPTION_ID'] = $sid
os.environ['AZURE_CLIENT_SECRET'] = $password
os.environ['AZURE_TENANT_ID'] = $tid
os.environ['AZURE_CLIENT_ID'] = $cid

In [49]:
saz_obj = saz()

### Template from Azure-Samples

In [44]:
url = "https://raw.githubusercontent.com/Azure-Samples/resource-manager-python-template-deployment/master/templates/template.json"

### Deploy with Template and Parameters

The sample template requires three parameters:

- **sshKeyData** which is ssh public key string
- **dnsLabelPrefix** which is unique DNS Name for the Storage Account 
- **vmName** which is virtual machine name to deploy

In [ ]:
saz_obj.arm.deploy(template = url, param = {"sshKeyData": "ssh-rsa AAAAB3...<skipped>... hroe.lee@simpleazure", 'dnsLabelPrefix':"simpleazure", 'vmName':'simpleazure-first-vm'})

### Termination (deleting resource group)

Deleting a resource group where deployments are made stops all services and deletes resources in the group.
Simple Azure uses prefixed group name 'saz' and the following function will delete the group.

In [67]:
saz_obj.arm.remove_resource_group()